In [153]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [154]:
df = spark.read.options(delimiter=";", inferSchema='True', header=True).csv("/home/mario/Downloads/10minutenwerte_only_100/produkt_zehn_min_tu_19900101_19991231_03366.txt")
df.printSchema()

root
 |-- STATIONS_ID: double (nullable = true)
 |-- MESS_DATUM: long (nullable = true)
 |-- QN: double (nullable = true)
 |-- PP_10: double (nullable = true)
 |-- TT_10: double (nullable = true)
 |-- TM5_10: double (nullable = true)
 |-- RF_10: double (nullable = true)
 |-- TD_10: double (nullable = true)



In [155]:
df.show(n=5)

+-----------+------------+---+-----+-----+------+-----+-----+
|STATIONS_ID|  MESS_DATUM| QN|PP_10|TT_10|TM5_10|RF_10|TD_10|
+-----------+------------+---+-----+-----+------+-----+-----+
|     3366.0|199001010000|1.0|972.6| -4.4|  -4.0| 94.0| -4.6|
|     3366.0|199001010010|1.0|972.5| -4.3|  -3.9| 94.0| -4.7|
|     3366.0|199001010020|1.0|972.4| -4.4|  -4.0| 94.0| -4.7|
|     3366.0|199001010030|1.0|972.3| -4.4|  -4.1| 94.0| -4.6|
|     3366.0|199001010040|1.0|972.3| -4.5|  -4.1| 94.0| -4.8|
+-----------+------------+---+-----+-----+------+-----+-----+
only showing top 5 rows



In [156]:
#df.withColumn("MESS_DATUM",df.MESS_DATUM.cast(StringType())) \
#  .show(truncate=False)

In [157]:
from datetime import datetime
from pyspark.sql.functions import col, udf, to_timestamp, year, month
from pyspark.sql.types import DoubleType, IntegerType, DateType, StringType
df = df.select("STATIONS_ID","MESS_DATUM","TT_10")
df = df.withColumn("MESS_DATUM",col("MESS_DATUM").cast(StringType()))
df = df.withColumn("STATIONS_ID",col("STATIONS_ID").cast(IntegerType()))
df = df.withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))

In [158]:
df.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- TT_10: double (nullable = true)



In [159]:
df.show(n=5)

+-----------+-------------------+-----+
|STATIONS_ID|         MESS_DATUM|TT_10|
+-----------+-------------------+-----+
|       3366|1990-01-01 00:00:00| -4.4|
|       3366|1990-01-01 00:10:00| -4.3|
|       3366|1990-01-01 00:20:00| -4.4|
|       3366|1990-01-01 00:30:00| -4.4|
|       3366|1990-01-01 00:40:00| -4.5|
+-----------+-------------------+-----+
only showing top 5 rows



Bauernregel:

„Wenn's im Dezember nicht wintert, sommert's im Juni auch nicht.“

Hierbei handelt es sich im Prinzip um die entgegengesetzte Regel zu der vorherigen:
Im Juni wird es nicht warm, wenn im Dezember kein richtiger Winter war.
Es stimmt, dass die These zu 65 Prozent eintrifft. Andersrum, wenn es also im Juni nicht „sommert“, wird es im Dezember nicht „wintern“,
stimmt die Bauernregel aber nicht. Warum das so ist, weiß keiner so genau.


In [160]:
df.describe().show()


+-------+-----------+------------------+
|summary|STATIONS_ID|             TT_10|
+-------+-----------+------------------+
|  count|     480758|            480758|
|   mean|     3366.0|-37.06368068758087|
| stddev|        0.0|209.83299914047205|
|    min|       3366|            -999.0|
|    max|       3366|              35.6|
+-------+-----------+------------------+



In [161]:
# we only need months June and Decemeber
df = df.filter((month("MESS_DATUM") == 6) | (month("MESS_DATUM") == 12))
df.show()

+-----------+-------------------+-----+
|STATIONS_ID|         MESS_DATUM|TT_10|
+-----------+-------------------+-----+
|       3366|1990-06-01 00:00:00|  6.7|
|       3366|1990-06-01 00:10:00|  6.9|
|       3366|1990-06-01 00:20:00|  6.2|
|       3366|1990-06-01 00:30:00|  5.8|
|       3366|1990-06-01 00:40:00|  5.8|
|       3366|1990-06-01 00:50:00|  6.0|
|       3366|1990-06-01 01:00:00|  6.1|
|       3366|1990-06-01 01:10:00|  6.0|
|       3366|1990-06-01 01:20:00|  5.9|
|       3366|1990-06-01 01:30:00|  5.6|
|       3366|1990-06-01 01:40:00|  5.7|
|       3366|1990-06-01 01:50:00|  6.3|
|       3366|1990-06-01 02:00:00|  6.5|
|       3366|1990-06-01 02:10:00|  6.8|
|       3366|1990-06-01 02:20:00|  5.8|
|       3366|1990-06-01 02:30:00|  5.3|
|       3366|1990-06-01 02:40:00|  5.5|
|       3366|1990-06-01 02:50:00|  5.5|
|       3366|1990-06-01 03:00:00|  6.5|
|       3366|1990-06-01 03:10:00|  7.3|
+-----------+-------------------+-----+
only showing top 20 rows



In [162]:
df = df.select("STATIONS_ID", year("MESS_DATUM").alias("year"),month("MESS_DATUM").alias("month"), "TT_10")
df.show()

+-----------+----+-----+-----+
|STATIONS_ID|year|month|TT_10|
+-----------+----+-----+-----+
|       3366|1990|    6|  6.7|
|       3366|1990|    6|  6.9|
|       3366|1990|    6|  6.2|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  6.0|
|       3366|1990|    6|  6.1|
|       3366|1990|    6|  6.0|
|       3366|1990|    6|  5.9|
|       3366|1990|    6|  5.6|
|       3366|1990|    6|  5.7|
|       3366|1990|    6|  6.3|
|       3366|1990|    6|  6.5|
|       3366|1990|    6|  6.8|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  5.3|
|       3366|1990|    6|  5.5|
|       3366|1990|    6|  5.5|
|       3366|1990|    6|  6.5|
|       3366|1990|    6|  7.3|
+-----------+----+-----+-----+
only showing top 20 rows



In [163]:
# calculate average temperature for each year+month
df = df.groupBy("year", "month").avg("TT_10")
df.show()

+----+-----+--------------------+
|year|month|          avg(TT_10)|
+----+-----+--------------------+
|1990|    6|  13.595092592592565|
|1990|   12| -11.698342803030272|
|1991|    6|  14.654583333333356|
|1992|    6|  -66.83006666666667|
|1993|    6|  14.659828465461315|
|1994|    6|   16.76222737819026|
|1993|   12|  2.4830121021963185|
|1995|   12| -1.2314964157706108|
|1994|   12|    2.02283535217586|
|1995|    6|  14.494441871236706|
|1996|    6|  17.114129273504336|
|1996|   12|  -4.925990752972258|
|1997|   12|  0.8322356630824369|
|1998|    6|   17.09474912485418|
|1997|    6|   16.27305555555556|
|1999|    6|  15.699606481481515|
|1998|   12| -1.1544822949350064|
|1999|   12|-0.18109441578829447|
+----+-----+--------------------+



In [164]:
df2 = df.filter(df.month == 12)
df2.show()

+----+-----+--------------------+
|year|month|          avg(TT_10)|
+----+-----+--------------------+
|1990|   12| -11.698342803030272|
|1993|   12|  2.4830121021963185|
|1995|   12| -1.2314964157706108|
|1994|   12|    2.02283535217586|
|1996|   12|  -4.925990752972258|
|1997|   12|  0.8322356630824369|
|1998|   12| -1.1544822949350064|
|1999|   12|-0.18109441578829447|
+----+-----+--------------------+



In [165]:
from pyspark.sql.functions import lit

df2 = df2.withColumnRenamed('avg(TT_10)', 'temperature_december')
df2 = df2.withColumnRenamed('year', 'year_december')
df2 = df2.withColumnRenamed('month', 'month_december')
df2 = df2.withColumn('year_june', df2.year_december + 1)
df2 = df2.withColumn('month_june', lit(6))
df2.show()

+-------------+--------------+--------------------+---------+----------+
|year_december|month_december|temperature_december|year_june|month_june|
+-------------+--------------+--------------------+---------+----------+
|         1990|            12| -11.698342803030272|     1991|         6|
|         1993|            12|  2.4830121021963185|     1994|         6|
|         1995|            12| -1.2314964157706108|     1996|         6|
|         1994|            12|    2.02283535217586|     1995|         6|
|         1996|            12|  -4.925990752972258|     1997|         6|
|         1997|            12|  0.8322356630824369|     1998|         6|
|         1998|            12| -1.1544822949350064|     1999|         6|
|         1999|            12|-0.18109441578829447|     2000|         6|
+-------------+--------------+--------------------+---------+----------+



In [166]:
df2 = df2.join(df, (df2.year_june == df.year) & (df2.month_june == df.month))
df2 = df2.withColumnRenamed('avg(TT_10)', 'temperature_june')
df2 = df2.drop("year", "month")
df2.show()

+-------------+--------------+--------------------+---------+----------+------------------+
|year_december|month_december|temperature_december|year_june|month_june|  temperature_june|
+-------------+--------------+--------------------+---------+----------+------------------+
|         1990|            12| -11.698342803030272|     1991|         6|14.654583333333356|
|         1993|            12|  2.4830121021963185|     1994|         6| 16.76222737819026|
|         1994|            12|    2.02283535217586|     1995|         6|14.494441871236706|
|         1995|            12| -1.2314964157706108|     1996|         6|17.114129273504336|
|         1997|            12|  0.8322356630824369|     1998|         6| 17.09474912485418|
|         1996|            12|  -4.925990752972258|     1997|         6| 16.27305555555556|
|         1998|            12| -1.1544822949350064|     1999|         6|15.699606481481515|
+-------------+--------------+--------------------+---------+----------+--------

In [167]:
df2.corr("temperature_december", "temperature_june", method="pearson")

0.4056745227573053

In [ ]:
#df.write.option("header",True).csv("/home/mario/Downloads/10minutenwerte/all_together_first_half")